# Ayudantía C06 - Geopandas

Esta ayudantía está basada en el laboratorio correspondiente del semestre 2019-1, el material de clases, y trabajos entregados. Trabajaremos con un archivo de emisiones GPS de cada recorrido del Transantiago, y shapes de sus paraderos y de las comunas de Santiago. Los archivos shape pueden ser de tres tipos: punto, línea, o polígono, y contienen información que se escribe en cinco archivos distintos. 

Veamos cómo se ven el shape de comunas de Santiago en un Geodataframe. Observemos que tiene una columna de geometría, que nos dice qué tipo de shape es, con su información correspondiente.

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as shapely

In [ ]:
zonas=gpd.read_file('zonas.shp',crs={'init':'epsg:4236'})
zonas = zonas.to_crs(4236)
zonas.head(5)

In [ ]:
zonas.plot()

El estándar de codificación con el que leemos nuestro archivo de paradas no entiende acentos del castellano, por lo que debemos quitarlas a mano.

In [ ]:
paradas=gpd.read_file('Paradas.shp',crs={'init':'epsg:4236'}, encoding='latin-1')

In [ ]:
paradas.plot()

In [ ]:
paradas.head(5)

In [ ]:
base = zonas.plot(figsize=(15,15), color='white', edgecolor='black')
paradas.plot(ax=base, marker='o', color='red', markersize=2);

Ahora revisemos el archivo de emisiones. Este archivo no es un Geodataframe, solamente un CSV que podemos convertir a Dataframe. Podrán imaginar la gran cantidad de datos que hay, por lo que la base de datos completa es gigante. Vamos a filtrar entonces la base de datos para trabajar con un solo recorrido.

In [ ]:
df=pd.read_csv("emisiones.csv")
def filtro(linea,sentido,df):
    df_filtrado=df.loc[lambda df: (df.line_code==linea)&(df.direction==sentido)]
    return df_filtrado

In [ ]:
linea='C01'
sentido='I'

df1=filtro(linea,sentido,df)
df1.head(5)

Podemos observar que este Dataframe tiene dos columnas con la información geográfica que necesitamos para convertirlo en un Geodataframe. Para esto usaremos el método Point de la librería Shapely.

El método zip recibe elementos iterables (nuestras columnas lat y lon, en este caso), y los agrupa en una tupla. Esta tupla se la pasamos al método Point, que crea objetos geográficos. Luego limpiamos nuestro Dataframe.

In [ ]:
df1 = df1.dropna(subset=['latitude', 'longitude']) #descartamos las filas con datos que no nos sirven
puntos=[shapely.Point(xy) for xy in zip(df1.longitude,df1.latitude)] #creamos instancias de la clase Point
df1=df1.drop(['longitude','latitude'],axis=1) #limpiamos el Dataframe
gdf1=gpd.GeoDataFrame(df1, crs={'init': 'epsg:4236'}, geometry=puntos) #Creamos el Geodataframe

In [ ]:
gdf1 #gdf1 es el geodataframe del recorrido filtrado.

In [ ]:
base = zonas.plot(figsize=(15,15), color='white', edgecolor='black')
gdf1.plot(ax=base, marker='o', color='red', markersize=2);

In [ ]:
lascondes = zonas.loc[zonas['COMUNA'] == 'LAS CONDES']
lascondes = lascondes.to_crs(4236)

Uno de los métodos más útiles de Geopandas es sjoin, que hace un "join" espacial de dos Geodataframes, y permite realizar operaciones entre ellos. Por ejemplo, vamos a crear un nuevo Geodataframe con todos los puntos del recorrido C01 que están dentro de la comuna de Las Condes.

In [ ]:
gdf2=gpd.sjoin(gdf1,lascondes,op='within')

In [ ]:
base = lascondes.plot(figsize=(15,15), color='white', edgecolor='black')
gdf2.plot(ax=base, marker='o', color='red', markersize=2);

Esta visualización nos recuerda que aunque el trazado se ve homogéneo desde lejos, en verdad solo es una dispersión de puntos GPS. Podríamos crear un objeto línea a partir de estos puntos, y así tener el trazado del recorrido que queremos.

Para ello, ordenemos el Geodataframe según la distancia desde el punto del despacho. Esta información está almacenada en la columna "distance_kms"

In [ ]:
puntos1=list(gdf1.sort_values(by='distance_kms').geometry)

In [ ]:
trazado1=shapely.LineString(puntos1)

Tenemos ahora un objeto LineString que podemos añadir como elemento geográfico al Dataframe filtrado del servicio seleccionado, y tener así un Geodataframe.

In [ ]:
trazado1